### Main notebook

In [1]:
import pandas as pd
import numpy as np
import json
import networkx as nx
from tqdm import tqdm
from compound import Compound
from reaction import Reaction
from graph import Graph
from data import Data

# read data from csv
cpds = pd.read_csv('../GNN_toxic/data/raw/compounds_final.csv', index_col=0) # containing toxicity
rxns = pd.read_csv('data/reactions_final.csv', index_col=0)
pairs = pd.read_csv('data/pairs_final.csv', index_col=0)
cofactors = pd.read_csv('data/original/cofactors_KEGG.csv')

# create class instances
data = Data()
graph = Graph(pairs=pairs)

In [2]:
# Create a Compound object for each row in the DataFrame and add it to the data
for index, row in cpds.iterrows():
    entry = row['Entry']
    name = row['Names']
    formula = row['Formula']
    mw = row['mol_weight']
    smiles = row['SMILES']
    is_cofactor = row['Entry'] in cofactors['Entry'].values
    is_toxic = row['toxic']

    compound = Compound(entry, name, formula, mw, smiles, is_cofactor, is_toxic)
    data.add_element('compound', compound)

# Create a Reaction object for each row in the DataFrame and add it to the data
for index, row in rxns.iterrows():
    entry = row['Entry']
    name = row['Names']
    compounds = row['Compound']
    enzyme = row['EC Number']

    reaction = Reaction(entry, name, compounds, enzyme)
    data.add_element('reaction', reaction)

In [3]:
# number of times a metabolite apperas on pairs dataset
graph.get_number_of_occurences(pairs)
graph.create_graph(data=data, pairs=pairs)

# nodes: 8591 
# edges: 30026


100%|██████████| 8591/8591 [00:00<00:00, 431303.68it/s]


In [4]:
graph.calculate_edge_mol_weight(data)
# graph.calculate_smiles_similarity(data)

100%|██████████| 30026/30026 [00:00<00:00, 383692.30it/s]


In [5]:
######### VALIDATION SET FROM nicepath ###########
test_cases = pd.read_csv('data/original/test_cases.csv')
test_cases['source'] = test_cases['Pathway '].apply(lambda x: x.split(',')[0])
test_cases['target'] = test_cases['Pathway '].apply(lambda x: x.split(',')[len(x.split(','))-1])
test_cases['paths_list'] = test_cases['Pathway '].apply(lambda x: x.split(','))
paths = graph.validate(test_cases, 'mol_weight')

Correct pathway predictions: 14
Correct pathway predictions (%): 28.0


In [8]:
p = graph.constrained_shortest_path('C00082', 'C01533', weight='mol_weight')
for i in p: print(i)

['C00082', 'C00822', 'C00355', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00811', 'C16828', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00822', 'C00355', 'C04045', 'C22038', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']
['C00082', 'C00822', 'C01693', 'C05604', 'C00355', 'C10447', 'C01197', 'C01494', 'C05619', 'C00482', 'C05610', 'C02325', 'C01533']


In [7]:
1/0

ZeroDivisionError: division by zero

In [ ]:
p = graph.constrained_shortest_path('C00082', 'C01533', weight='mol_weigh')
p

In [9]:
from itertools import chain

def has_duplicates(lst):
    return len(lst) != len(set(lst))

p = graph.shortest_simple_paths('C00082', 'C01533', weight='mol_weight')

smiles_sim = []

for j in p:
    rxns_list = []
    sum = 0
    compound_list = j
    for i in range(len(compound_list) - 1):
        current_compound = compound_list[i]
        next_compound = compound_list[i + 1]
        # sum = sum + graph.G.edges()[current_compound, next_compound]['smiles_similarity']
        # if graph.G.edges()[current_compound, next_compound]['smiles_similarity'] == 0:
        #     sum = sum + graph.G.edges()[next_compound, current_compound]['smiles_similarity']
        rxns_list.append(pairs[(pairs['source']==current_compound) & (pairs['target']==next_compound)]['Reaction'].values)
        rxns_list.append(pairs[(pairs['source']==next_compound) & (pairs['target']==current_compound)]['Reaction'].values)

    smiles_sim.append(sum)
    rxns_list = [arr for arr in rxns_list if arr.any()]
    rxns_list = list(chain.from_iterable(rxns_list))
    has_duplicate = has_duplicates(rxns_list)
    print(has_duplicate)

False
True
False
False
True
False
True
False
True
False


In [ ]:
p

In [ ]:
idx = smiles_sim.index(min(smiles_sim))

print(idx)

In [ ]:
from itertools import islice

next(islice(p, idx, None))

In [ ]:
test_cases['Pathway '].iloc[0]

In [ ]:
paths.iloc[0]['Pathway']

In [ ]:
1/0

In [ ]:
for i in graph.shortest_simple_paths('C00082', 'C01533', length=10, weight='mol_weight'):
    print(i)

In [ ]:
test_cases

### Try cluster graph nodes

### Try similarity based on SMILES

In [ ]:
smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C00323').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))


smiles1 = data.get_compound_by_id('C00223').smiles
smiles2 = data.get_compound_by_id('C12096').smiles

from rdkit import Chem
from rdkit import DataStructs

ms = [Chem.MolFromSmiles(smiles1), Chem.MolFromSmiles(smiles2)]
fs = [Chem.RDKFingerprint(x) for x in ms]
s = DataStructs.FingerprintSimilarity(fs[0], fs[1])
print('Similarity: '+str(s))